In [1]:
!pip install opencv-python
!pip install pdf2image
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.86.1-0ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [2]:
from PIL import Image
import pytesseract

from pdf2image import convert_from_path
def op_ocr(li):
    s=''
    images = convert_from_path(li)
    for i in images:
        text = pytesseract.image_to_string(i, lang="eng")
        s=s+text
    return(s)

        

In [4]:
def defan(con):
    regex=r"VS.|vs."
    #regex2=r"Defendan"
    match = re.search(regex, con)
    #match2=re.search(regex2,con)
    defandant=con[match.end():]
    return defandant

In [6]:
def case_nos(con):
    pattern=r"ADJ[0-9]+"
    cases=re.findall(pattern,con)
    new_con=re.sub(pattern, '', con)
    #new_con=re.sub(pattern, '', con)
    return cases,new_con

In [5]:
def applicant(con):
    app_regex=r"Applic"
    case=r"Case .*"
    match_app = re.search(app_regex, con)
    case_match=re.search(case,con)
    applicant=con[case_match.end():match_app.start()]
    return applicant.strip('\n')

In [7]:
words=['OPINION','AND','DECISION','AFTER','RECONSIDERATION','PETITION','RECONSIDERATION','REMOVAL','DECISION AFTER REMOVAL','ORDER','GRANTING', 'FOR']
def clear_fucking_mess(con):
    for i in words:
        con = con.replace(i,'')
    return con
    

In [3]:
def sabkaammaihikarukya(content,filename):
    con=extract_sucks(content)
    new,district=extract_location(con)
#print(new)
    cases,new_d=case_nos(new)
    appl=applicant(new_d)
#print(cases)
    #print(district)
    #print(new_d)
#print(applicant(new_d))
    new_fucking_point=clear_fucking_mess(new_d)
#print(new_fucking_point)
    defandant=defan(new_fucking_point).strip("\n")
    #print(defandant)
    #print(new_fucking_point)
    code_reg=r"section [0-9]+"
    code = re.findall(code_reg, content)
    code_set=set(code)
    code=list(code_set)
    
    class Output:
        def __init__(self, defandant, appl, code,cases,district,filename):
            self.defandant = defandant
            self.appl = appl
            self.code = code
            self.cases=cases
            self.district=district
            self.filename=filename
            
    out=Output(defandant, appl, code,cases,district,filename)
    import json
    output_str=json.dumps(out.__dict__)
    #convert string to  object
    data = json.loads(output_str)
    return data

In [16]:
import os
import urllib.request
from flask import app
from flask import Flask, request, redirect, jsonify
from werkzeug.utils import secure_filename


ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
# using flask_restful
from flask import Flask, jsonify, request,render_template
from flask_restful import Resource, Api
  
# creating the flask app
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'test_Api'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
# creating an API object
api = Api(app)
  
# making a class for a particular resource
# the get, post methods correspond to get and post requests
# they are automatically mapped by flask_restful.
# other methods include put, delete, etc.
class Extract_v1(Resource):
  
    # corresponds to the GET request.
    # this function is called whenever there
    # is a GET request for this resource
    def get(self):
        
       return "jd"

  
    # Corresponds to POST request
    def post(self):
        print(request.files)
        if 'file' not in request.files:
            print("error")
            resp = jsonify({'message' : 'No file part in the request'})
            resp.status_code = 400
            return resp
        file = request.files['file']
        if file.filename == '':
            resp = jsonify({'message' : 'No file selected for uploading'})
            resp.status_code = 400
            return resp
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            resp = jsonify({'message' : 'File successfully uploaded'})
            resp.status_code = 201
            return resp
        else:
            resp = jsonify({'message' : 'Allowed file types are txt, pdf, png, jpg, jpeg, gif'})
            resp.status_code = 400
            return resp
          
        data = request.get_json()     # status code
        return jsonify({'data': data}), 201
  
  

  
# adding the defined resources along with their corresponding urls
api.add_resource(Extract_v1, '/api/extract_v1/')

  
  
# driver function
if __name__ == '__main__':
  
    app.run(debug = True)

 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/werkzeug/serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1978031/1657655738.py", line 75, in <module>
    app.run(debug = True)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1188, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/opt/conda/lib/python3.7/site-packages/werkzeug/serving.py", line 1062, in run_simple
    s = prepare_socket(hostname, port)
  File "/opt/conda/lib/python3.7/site-packages/werkzeug/serving.py", line 930, in prepare_socket
    sys.exit(1)
SystemExit: 1

During handling of the above exception, an

TypeError: object of type 'NoneType' has no len()

In [14]:
!pip install flask-restful


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 9.0 MB/s eta 0:00:00
